Spinorama plot with Altair

In [1]:
import math
import pandas as pd
import altair as alt
import numpy as np
import flammkuchen as fl

import sys,os,os.path
sys.path.append(os.path.expanduser('../src'))

from spinorama.graph import graph_params_default, graph_freq, graph_spinorama
from spinorama.load import filter_graphs

df_all = fl.load('../cache.parse_all_speakers.h5')

In [4]:
from spinorama.load import filter_graphs, load_normalize

df_klippel = df_all['A']['Adam S2V']['ASR']['asr']
spl_H = df_klippel['SPL Horizontal_unmelted']
spl_V = df_klippel['SPL Vertical_unmelted']
df_computed = load_normalize(filter_graphs('Adam S2V', spl_H, spl_V))

Compare Klippel graphs with computed ones.

In [5]:
graph_spinorama(df_klippel['CEA2034'], graph_params_default) | graph_spinorama(df_computed['CEA2034'], graph_params_default)

alt.HConcatChart(...)

In [6]:
graphs = {}
for g in ('Early Reflections', 'Vertical Reflections', 'Horizontal Reflections'):
    graphs[g] = graph_freq(df_klippel[g], graph_params_default) | graph_freq(df_computed[g], graph_params_default)

In [7]:
graphs['Early Reflections']

alt.HConcatChart(...)

In [8]:
graphs['Vertical Reflections']

alt.HConcatChart(...)

In [9]:
graphs['Horizontal Reflections']

alt.HConcatChart(...)

In [30]:
alt.data_transformers.disable_max_rows()
    
nearest = alt.selection(
    type="single", nearest=True, on="mouseover", fields=["Freq"], empty="none"
)
   
colors = [
    "#5c77a5",
    "#dc842a",
    "#c85857",
    "#89b5b1",
    "#71a152",
    "#bab0ac",
    "#e15759",
    "#b07aa1",
    "#76b7b2",
    "#ff9da7",
]

uniform_color_pair = {
    # regression
    "Linear Regression": colors[0],
    "Band ±1.5dB": colors[1],
    "Band ±3dB": colors[1],
    # PIR
    "Estimated In-Room Response": colors[0],
    # spin
    "On Axis": colors[0],
    "Listening Window": colors[1],
    "Early Reflections": colors[2],
    "Sound Power": colors[3],
    "Early Reflections DI": colors[4],
    "Sound Power DI": colors[5],
    # reflections
    "Ceiling Bounce": colors[1],
    "Floor Bounce": colors[2],
    "Front Wall Bounce": colors[3],
    "Rear Wall Bounce": colors[4],
    "Side Wall Bounce": colors[5],
    #
    "Ceiling Reflection": colors[1],
    "Floor Reflection": colors[2],
    #
    "Front": colors[1],
    "Rear": colors[2],
    "Side": colors[3],
    #
    "Total Early Reflection": colors[7],
    "Total Horizontal Reflection": colors[8],
    "Total Vertical Reflection": colors[9],
    # SPL
    "10°": colors[1],
    "20°": colors[2],
    "30°": colors[3],
    "40°": colors[4],
    "50°": colors[5],
    "60°": colors[6],
}

uniform_color_domain = list(uniform_color_pair.keys())
uniform_color_range = list(uniform_color_pair.values())
uniform_scale = alt.Scale(domain=uniform_color_domain, range=uniform_color_range)
    
def filtered_scale(dfu):
    measurements = sorted(set(dfu.Measurements))
    filtered_domain = [d for d in uniform_color_domain if d in measurements]
    filtered_range = [
        uniform_color_pair[d] for d in uniform_color_domain if d in measurements
    ]
    return alt.Scale(domain=filtered_domain, range=filtered_range)

    
def graph_spinorama_exp(dfu, graph_params):
    xmin = graph_params["xmin"]
    xmax = graph_params["xmax"]
    ymin = graph_params["ymin"]
    ymax = graph_params["ymax"]
    if xmax == xmin:
        logger.error("Graph configuration is incorrect: xmin==xmax")
    if ymax == ymin:
        logger.error("Graph configuration is incorrect: ymin==ymax")
    # add selectors
    selectorsMeasurements = alt.selection_multi(fields=["Measurements"], bind="legend")
    scales = alt.selection_interval(bind="scales")
    scales_di = alt.selection_interval(bind="scales")
    # main charts
    xaxis = alt.X(
        "Freq:Q",
        title="Freqency (Hz)",
        scale=alt.Scale(type="log", base=10, nice=False, domain=[xmin, xmax]),
        axis=alt.Axis(format="~s"),
    )
    yaxis = alt.Y(
        "dB:Q",
        title="Sound Pressure (dB)",
        scale=alt.Scale(zero=False, domain=[ymin, ymax]),
    )
    # why -10?
    di_yaxis = alt.Y(
        "dB:Q",
        title="Sound Pressure DI (dB)",
        scale=alt.Scale(zero=False, nice=False, domain=[-5, ymax - ymin - 5]),
        axis=alt.Axis(
            grid=True, tickCount=10, labelExpr="datum.value >15 ? null : datum.label"
        ),
    )
    color = alt.Color(
        "Measurements", scale=filtered_scale(dfu), type="nominal", sort=None
    )
    opacity = alt.condition(selectorsMeasurements, alt.value(1), alt.value(0.2))

    line = (
        alt.Chart(dfu)
        .mark_line()
        .transform_filter(
            alt.FieldOneOfPredicate(
                field="Measurements",
                oneOf=[
                    "On Axis",
                    "Listening Window",
                    "Early Reflections",
                    "Sound Power",
                ],
            )
        )
        .encode(x=xaxis, y=yaxis, color=color, opacity=opacity)
    )
    
    circle = (
        alt.Chart(dfu)
        .mark_circle(size=100)
        .transform_filter(
            alt.FieldOneOfPredicate(
                field="Measurements",
                oneOf=[
                    "On Axis",
                    "Listening Window",
                    "Early Reflections",
                    "Sound Power",
                ],
            )
        )
        .encode(
            x=xaxis,
            y=yaxis,
            color=color,
            opacity=alt.condition(nearest, alt.value(1), alt.value(0)),
            tooltip=["Measurements", "Freq", "dB"],
        )
    )
    
    di = (
        alt.Chart(dfu)
        .mark_line()
        .transform_filter(
            alt.FieldOneOfPredicate(
                field="Measurements", oneOf=["Early Reflections DI", "Sound Power DI"]
            )
        )
        .encode(x=xaxis, y=di_yaxis, color=color, opacity=opacity)
    )
    circle_di = (
        alt.Chart(dfu)
        .mark_circle(size=100)
        .transform_filter(
            alt.FieldOneOfPredicate(
                field="Measurements", oneOf=["Early Reflections DI", "Sound Power DI"]
            )
        )
        .encode(
            x=xaxis,
            y=di_yaxis,
            color=color,
            opacity=alt.condition(nearest, alt.value(1), alt.value(0)),
            tooltip=["Measurements", "Freq", "dB"],
        )
    )
            
    # assemble elements together
    spin = (
        alt.layer(
            (circle + line).add_selection(scales),
            (circle_di + di).add_selection(scales_di),
        )
        .resolve_scale(y="independent")
        .add_selection(selectorsMeasurements)
        .add_selection(nearest)
        .properties(width=graph_params["width"], height=graph_params["height"])
    )
        
    return spin                                                                                                 

In [31]:
graph_spinorama_exp(df_klippel['CEA2034'], graph_params_default)

alt.LayerChart(...)